# Getting started with operators

Qiskit provides sophisticated functionality with operators: [The "Operator Flow" layer in Aqua](https://qiskit.org/documentation/apidoc/qiskit.aqua.operators.html). In this tutorial notebook we will get familiar with the main building blocks of the operator flow layer:

1. State functions
2. Operators and evolutions
3. Measurements and expectations

This tutorial assumes basic familiarty with qubits, quantum superposition and quantum circuits.

There are a few important underlying mathematical concepts:

* An n-qubit State function is a complex function over n binary variables, which we will often refer to as n-qubit binary strings. For example, the traditional quantum “zero state” is a 1-qubit state function, with a definition of f(0) = 1 and f(1) = 0.
* An n-qubit Operator is a linear function taking n-qubit state functions to n-qubit state functions. For example, the Pauli X Operator is defined by f(Zero) = One and f(One) = Zero. Equivalently, an Operator can be defined as a complex function over two n-qubit binary strings, and it is sometimes convenient to picture things this way. By this definition, our Pauli X can be defined by its typical matrix elements, f(0, 0) = 0, f(1, 0) = 1, f(0, 1) = 1, f(1, 1) = 0.
* An n-qubit Measurement is a functional taking n-qubit State functions to complex values. For example, a Pauli Z Measurement can be defined by f(Zero) = 0 and f(One) = 1.

The definitions of these concepts mostly follows the ones in the first two chapters of the book ["The Theory of Quantum Information" by John Watrous](https://cs.uwaterloo.ca/~watrous/TQI/), but don't worry if you don't get them at the first glance: This tutorial is all about getting intuition behind them and the structure of the operator flow layer.

First, let us import the necessary components:

In [12]:
from qiskit.aqua.operators import *
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np

## State functions

Let us start with a single qubit. As we know, a pure state of a qubit is, in general, described as a superposition of basis states $\vert \psi \rangle = \alpha \vert 0\rangle + \beta \vert 1 \rangle$ with complex coefficients $\alpha$ and $\beta$. We can think of this superposition as some mathematical object which puts into correspondence the ket $\vert 0\rangle$ with $\alpha$ and $\vert 1\rangle$ with $\beta$. This is exactly what the state functions in Aqua are. For example the well known minus state $\vert \psi_{-}\rangle=\frac{1}{\sqrt{2}}\vert 0 \rangle - \frac{1}{\sqrt{2}}\vert 1\rangle$ can be thought of as a function mapping the basis ket $\vert 0\rangle$ to $\frac{1}{\sqrt{2}}$ and the basis ket $\vert 1\rangle$ to $-\frac{1}{\sqrt{2}}$. Basis states themselves are particular cases of state sunctions, e.g. the basis state $\vert 0\rangle$ can be thought of as a function mapping the ket $\vert 0\rangle$ to the number $1$ and the ket $\vert 1\rangle$ to the number $0$.

In Aqua, the class `StateFn` can be used to create state functions. Let us create the 1-qubit state function corresponding to the state $\vert \psi_{-}\rangle$

In [7]:
amplitudes = [1, -1]
normalization_coefficient = 1 / np.sqrt(2)
psi_minus = StateFn(amplitudes, coeff=normalization_coefficient)

To access the mapping represented by this state function we can use the function `.eval()`

In [8]:
print(f"The ket |0> is mapped to {psi_minus.eval('0')}")
print(f"The ket |1> is mapped to {psi_minus.eval('1')}")

The ket |0> is mapped to (0.7071067811865475+0j)
The ket |1> is mapped to (-0.7071067811865475+0j)


TODO: add a sentence about normalization (Aqua state functions attempt to be as general as possible, and be just abstract mathematical object with or without relation to quantum states)

If we look at the type of the object `psi_minus`

In [18]:
type(psi_minus)

qiskit.aqua.operators.state_fns.vector_state_fn.VectorStateFn

we can see that it is `VectorStateFn`. This is not much of a surprise, since we built the state function based on a vector of amplitudes. In how many ways can we create a state function? One way is obviously with a vector of complex amplitudes as done above. Another way is by specifying the mapping explicitly with a python dictionary

In [11]:
mapping = {'0': 1, '1': -1}
StateFn(mapping, coeff=normalization_coefficient)

DictStateFn({'0': 1, '1': -1}, coeff=0.7071067811865475, is_measurement=False)

TODO: add something about the fact that in DictStateFn you can omit the zero amplitudes?

Yet another way - with a quantum circuit. TODO: add one or two sentences of explanations....

In [16]:
qc = QuantumCircuit(1)
qc.h(0)
qc.z(0)
StateFn(qc)

CircuitStateFn(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000021C9019F988>, coeff=1.0, is_measurement=False)

The class `StateFn` is a convenient way to create state function objects with all supported ways, however the underlying classes `VectorStateFn`, `DictStateFn`, `CircuitStateFn` etc. can be used directly as well, e.g. `DictStateFn({'0': 1, '1': -1})`. The full list of possible ways to build a state function in Aqua can be studied in the documentation. One important way to do this is by operators, which we will cover briefly when we introduce the measurements.

TODO: add a note about pre-built one-qubit stte functions in operator globals....

TODO: multiqubit states and tensor product (talk about TensoredOp as well?)....

TODO: State functions support various operations. Some important ones are sample, to_matrix, etc?

## Operators

Now as we can build state functions we can define operations to transform them. Aqua operators do exactly this. Let us define an operator which takes the $\vert - \rangle$ state to the basis state $\vert 1\rangle$. As you know we could accomplish this with a Hadamard gate if we were building a quantum circuit. Let us define an operator based on such a quantum circuit

In [19]:
qc_hadamard = QuantumCircuit(1)
qc_hadamard.h(0)
op_hadamard = CircuitOp(qc_hadamard)

To make the operator `op_hadamard` act on the state `Minus` we need to chain them together. The official term for this in Aqua is _composition_. The symbol `@` can be used to compose the operator with the state function, or the `.compose()` function: they are equivalent

In [26]:
(op_hadamard @ Minus).eval().sample()

{'1': 1.0}

In [27]:
op_hadamard.compose(Minus).eval().sample()

{'1': 1.0}

The state functions and operators have a similar interface. TODO: a few sentences about lazy composition, the type of the object created as a result of composition and the eval function....

TODO: other ways to create operators (one important class - Puli operators)...

TODO: standard operators in operator globals....

## Evolutions

TODO:

TODO: building hamiltonians with operators (talk about SummedOp here as well)....

## Measurements and Expectations

TODO:

## Example

TODO: provide a simple end to end example of initializing some state, evolving it with some hamiltonian and measuring the expectation value of something (e.g. the hamiltonian itself)....